# Machine Learning Regression und Klassifikation Vertiefung

## TEIL A: Regression mit Hauspreisberechnung

<div style="padding: 5px; border: 5px solid #a10000ff;">

**Hinweis:** In den Codezellen sind jeweils einige Codeteile nicht programmiert. Diesen Code müssen Sie ergänzen. Die jeweiligen Stellen sind mit einem Kommentar und dem Keyword **TODO** vermerkt und z.T. Stellen mit ... markiert.

Ausserdem gibt es einige assert Statements. Diese geben einen Fehler aus, sollte etwas bei Ihrer Programmierung nicht korrekt sein.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np


In diesem Dataset wurden verschiedene Eigenschaften von Liegenschaften erfasst. 

Dabei soll nun von den Eigenschaften auf den Hauspreis geschlossen werden. Der Hauspreis ist somit die **Zielvariable** oder engl. *Target*, ähnlich dem Label in der Klassifikation.

Die Berechnungen des Hauspreises, werden wir mit einem Regressionsmodell machen.

Das Dataset das wir benutzten, ist das California Housing Dataset. **Dieses ist bereits vorbereitet**:
https://www.kaggle.com/datasets/camnugent/california-housing-prices

In [ ]:
# Wir laden den Datensatz in ein Pandas DataFrame und zeigen die ersten Zeilen an
df_housing = pd.read_csv("./data/housing.csv")
df_housing

### Aufgabe 1

Sie haben sich sicherlich die Features im Dataframe angeschaut. Machine Learning Modelle benötigen die Daten als Zahlen um diese im Features Space abbilden zu können. Jedoch haben wir mit ocean_proximity ein Feature das Kategorische Daten enthält.

**Frage:** Um welche Art von Skalentyp handelt es sich? Wie übertragen wir ein solches Feature in einen Feature Space?

<br>
<details>
<summary><b>Lösung: Klicke hier für die Lösung.</b></summary>

Es handelt sich um eine Nominalskala. Die Ordnung ist nicht klar gegeben. Es ist z.B. nicht klar ob Island näher am Ozean ist wie Near Ocean zum Beispiel.

Diese können wir mit dem sogenannten One-Hot-Encoding in einen mathematischen Raum übertragen. Dies geschieht indem wir für jede Kategorie eine neue Dimension anlegen und dort eine 1 vermerken wenn die Kategorie zutrifft und bei allen anderen eine 0. Wir nutzen dazu den One-Hot-Encoder von Scikit-learn.

Zusätzlich entfernen wir noch alle Data Samples die leere Werte haben.

</details>


Wir listen nun einmal alle Arten von Werten die ocean_proximity haben kann.

In [ ]:
# Wir verwenden OneHotEncoder aus sklearn.preprocessing
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# TODO Konfiguriere das One-Hot-Encoding auf der Spalte 'ocean_proximity' indem du das DataFrame df_housing mit der Spaltenangabe als Parameter einfügst. Beispiel: ohe.fit(df_iris[['petal length (cm)']])
ohe.fit(...)

# Wir erstellen ein neues DataFrame mit den kodierten Spalten und fügen sie dem ursprünglichen DataFrame hinzu. Danach entfernen wir die ursprüngliche Spalte 'ocean_proximity'.
df_housing_encoded = pd.concat([df_housing, pd.DataFrame(ohe.transform(df_housing[['ocean_proximity']]), columns=ohe.get_feature_names_out(['ocean_proximity']))], axis=1)
df_housing_encoded.drop('ocean_proximity', axis=1, inplace=True)

# Wir entfernen Zeilen mit fehlenden Werten, da diese nicht für das Training des Modells verwendet werden können
df_housing_encoded.dropna(inplace=True)

df_housing_encoded

### Aufgabe 1.1 : Verleiche nun die Ausgabe hier mit dem initialen DataFrame weiter oben. Wie viele Spalten sind nun dazugekommen und welche wurde entfernt?

### Skalierung

Wir möchten nun noch die Daten normalisieren. Dies hilft einigen Modellen zum Beispiel künstlichen Neuronalen Netzwerken schneller zu optimieren und zu lernen.
Wir wenden die min-max-Skalierung an. Das heisst alle Features haben danach einen minimalen Wert von 0 und einen maximalen Wert von 1.

**Führen Sie die nächste Zelle aus:**

$scaled\_value = \frac{value-min}{max - min}$

In [ ]:
 # Normalize numerical features with min max scaling

# Identify numerical features
numerical_features = df_housing_encoded.select_dtypes(include=['float64', 'int64']).columns

# Apply min-max scaling
for feature in numerical_features:
    min_value = df_housing_encoded[feature].min()
    max_value = df_housing_encoded[feature].max()
    df_housing_encoded[feature] = (df_housing_encoded[feature] - min_value) / (max_value - min_value)

df_housing_encoded

# prüfen ob die numerischen Features korrekt normalisiert wurden
assert (df_housing_encoded[numerical_features].min().min() >= 0) and (df_housing_encoded[numerical_features].max().max() <= 1), "Die numerischen Features wurden nicht korrekt normalisiert."

### Aufgabe 2

Wir unterteilen das Dataset in ein Trainings und Testteil.
Dabei nehmen wir 80% Trainingsdaten und 20% Testdaten.

Lassen Sie die nächste Zelle laufen und geben Sie darunter aus, wie viele Trainings- und Testdaten Sie haben. **Tipp:** Nutzen Sie .shape oder .len()

In [ ]:
# Zusatzinfos: Wir Unterteile das Dataset in Trainigns- und Testdaten. Die Spalte 'median_house_value' ist die Zielvariable, die wir vorhersagen möchten. 
# Deshalb wird sie von den Features getrennt. Wir entfernen die Zielvariable aus den Features bei der Parameterübergabe mit df_housing_encoded.drop('median_house_value', axis=1) und benutze sie als zweiten Parameter in der train_test_split Funktion.

X_housing_train, X_housing_test, y_housing_train, y_housing_test = train_test_split(df_housing_encoded.drop('median_house_value', axis=1), df_housing_encoded['median_house_value'], test_size=0.2, random_state=42)


Die **Features** sind nun in dem Variablen mit X beginnend gespeichert. Die **Targets** in den Variablen mit y beginnend.

In [ ]:
#TODO Zeigen Sie an wie viele Trainings- und Testdaten Sie haben


In [ ]:
# Tests zur Überprüfung der Aufteilung der Daten in Trainings- und Testdaten
assert X_housing_train.shape[0] == 16346, f"Erwartete Anzahl Trainingsdaten: 16346, aktuell sind es: {X_housing_train.shape[0]}"
assert X_housing_test.shape[0] == 4087 , f"Erwartete Anzahl Testdaten: 4087, aktuell sind es: {X_housing_test.shape[0]}"

# Prüfe ob median_house_value aus den Features entfernt wurde
assert 'median_house_value' not in X_housing_train.columns, "median_house_value wurde nicht aus den Features entfernt."

### Aufgabe 3

1. Nutzen Sie die MLPRegressor Klasse um ein Modell zu instantieren. Die Klasse wurde bereits am Anfang importiert. Sie können die gleichen Parameter verwenden wie in Aufgabe 7 beim MLPClassifier.
2. Trainieren Sie nun das Modell mit dem Aufruf der fit(Trainingsdaten, Targets) Methode.

Optional: Weitere Infos zur MLPRegressor Klasse: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

In [ ]:
# Wir erstellen nun ein Regressionsmodell bestehend aus mehreren Perzeptronen (MLPRegressor)
mlp_regressor = MLPRegressor(hidden_layer_sizes=(10,), max_iter=500, random_state=42)

# TODO Trainieren Sie das Modell mit den Trainignsdaten als erstes Argument und den zugehörigen Labels als zweites Argument. Tipp: benutzen Sie die X_housing_train und y_housing_train Variablen.
mlp_regressor.fit(..., ...)

### Aufgabe 4

Evaluieren Sie nun ihr Modell mit den Testdaten. Dieses Mal können wir aber nicht die Accuracy nutzen, da diese nur für Klassifikationen geeignet ist.
Wir nutzen stattdessen den Root-Mean-Squared-Error. Dieser wird wie folgt berechnet:

- $y$: Echtes Label
- $\hat{y}$: Voraussage des Modells

$\text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2}$

In Prosa geschieht hier folgendes:
Für jedes Data Samples im Testdatenset wird das echte Label minus der Voraussage gerechnet. Dieses Ergebnis wird quadriert. Danach wird die Summe über alle diese quadrierten "Fehler" berechnet und geteilt durch die Anzahl Samples gerechnet. Somit der Mittelwert des quadrierten Fehlers. Zuletzt ziehen wir noch die Wurzel damit das Ergebnis besser interpretierbar wird, bezüglich der Grössenordnung.

Vervollständigen Sie den Code um den MSE zu berechnen.

In [ ]:
# Wir berechnen nun mit dem Modell die Vorhersagen für die Testdaten als einzigen Parameter in der predict-Methode
y_housing_pred = mlp_regressor.predict(X_housing_test)

#TODO  Wir berechnen den Root Mean Squared Error (RMSE) auf dem Testset
mse_test = np.sqrt(np.sum((... - ...)**2) / len(...))


In [ ]:
# Beispiel Vorhersage des Preises für ein einzelnes Haus aus dem Testset

y_housing_pred_single = mlp_regressor.predict(X_housing_test[:1])
print(f'Der berechnete Hauswert beträgt: {y_housing_pred_single[0]:.2f}')

y_pred_scaled = y_housing_pred_single * (df_housing['median_house_value'].max() - df_housing['median_house_value'].min()) + df_housing['median_house_value'].min()
print(f'Der berechnete Hauswert im Originalmaßstab beträgt: {y_pred_scaled[0]:.2f}')

### Aufgabe 5

Wir zeigen nun in einem Scatter Plot noch einige zufällige Datenpunkte an, wobei wir vergleichen möchten was der echte Hauspreis ist und was unser Modell berechnet hat.
Lassen Sie die nächste Code Zelle laufen und beantworten Sie die folgende Frage.

**Frage**
Woran erkennt man einen kleinen Fehler des Modells und wie einen grossen?


In [ ]:
# Plotte die Vorhersagen des Modells gegen die tatsächlichen Werte nutze aber nur 50 zufällige Datenpunkte und zeichne den Fehler als Linie ein

random_indices = np.random.choice(len(y_housing_test), size=50, replace=False)
y_housing_pred_sampled = y_housing_pred[random_indices]
y_housing_test_sampled = y_housing_test.iloc[random_indices]


plt.figure(figsize=(15, 6))
plt.scatter(range(len(y_housing_pred_sampled)), y_housing_pred_sampled, color='red', label='Berechnete Werte')
plt.scatter(range(len(y_housing_test_sampled)), y_housing_test_sampled, color='blue', label='Tatsächliche Werte')
for i in range(len(y_housing_pred_sampled)):
    plt.plot([i, i], [y_housing_pred_sampled[i], y_housing_test_sampled.iloc[i]], color='gray', linestyle='--', linewidth=0.5)
plt.xlabel('Testdaten Index')
plt.ylabel('Median Hauswert (normalisiert)')
plt.title('Vorhersagen vs Tatsächliche Werte des Hauswerts')
plt.legend()
plt.show()

## Kontrollfragen: Regression



**Kontrollfrage 3**

Was ist der Output einer Regression und wie verhält sich dieser im Vergleich zu der Klassifikation?


**Kontrollfrage 4**

Wie können jategorische Daten auch für ein Regressionsmodell nutzbar gemacht werden?
